In [1]:
import tensorflow as tf
import src.infovae as infovae

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
nn = infovae.InfoVAE(12, 16, 4)

x = tf.placeholder(shape=[None, 32, 32, 1], dtype=tf.float32)
x_ = nn(x)

recon_loss, latent_loss = nn.make_losses(x, x_)
loss = recon_loss+latent_loss

In [4]:
train_summaries = [
    tf.summary.scalar('train/loss/recon', recon_loss),
    tf.summary.scalar('train/loss/latent', latent_loss),
    tf.summary.histogram('latents', nn.z),
    tf.summary.image('train/recon', x_),
]
test_summaries = [
    tf.summary.scalar('test/loss/recon', recon_loss),
    tf.summary.scalar('test/loss/latent', latent_loss),
    tf.summary.image('test/recon', x_),
]

train_merged = tf.summary.merge(train_summaries)
test_merged = tf.summary.merge(test_summaries)

In [5]:
opt = tf.train.AdamOptimizer()
train_step = opt.minimize(loss)
saver = tf.train.Saver()

In [6]:
logdir = '/tmp/infovae/4'
!rm -rf {logdir}

In [7]:
epochs = 50

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(logdir, sess.graph)
    
    for i in range(epochs*1000):
        batch_x, _ = mnist.train.next_batch(50)
        _, train_summ = sess.run([train_step, train_merged], 
                                 feed_dict={x: infovae.InfoVAE.preprocess(batch_x)})
        
        if i % 10 == 0:
            writer.add_summary(train_summ, i)
        
        if i % 100 == 0:
            L, test_summ = sess.run([loss, test_merged], 
                                                feed_dict={x: 
                        infovae.InfoVAE.preprocess(mnist.test.images[:100, ...])})
            print('\rStep: {} Loss: {}'.format(i, L), end='', flush=True)
            writer.add_summary(test_summ, i)
    save_path = saver.save(sess, logdir+"/infovae.ckpt")

Step: 49900 Loss: 0.07996300607919693

In [9]:
save_path

'/tmp/infovae/4/infovae.ckpt'